In [ ]:
from ipywidgets import Text, ToggleButton, ToggleButtons, FloatSlider, HBox, Label, VBox
from ipywidgets import IntSlider, Layout, Button, AppLayout, IntProgress, Box, Textarea

import matplotlib.pyplot as plt
import pandas as pd
import random
import copy
import math
import time

## Implementacja chromosomu

#### o __init__ - konstruktor
- `bits` - liczba bitów chromosomu, liczba całkowita
- `binary` - chromosom w reprezentacji binarnej, tablica jednowymiarowa `ndarray`
- `decimal` - chromosom w reprezentacji dziesiętnej, liczba całkowita

#### o setBinary - implementacja binarnej reprezentacji chromosomu + konfiguracja dokładności
- `precision` - dokładność do określonej liczby cyfr znaczących, liczba całkowita

#### o setDecimal - implementacja rozkodowania chromosomu - konwersji  z reprezentacji binarnej do reprezentacji dziesiętnej
- `binary` - chromosom w reprezentacji binarnej, tablica jednowymiarowa `ndarray`
- `decimal` - chromosom w reprezentacji dziesiętnej, liczba całkowita

In [ ]:
class Chromosome:
    def __init__(self, precision):
        self.precision = precision
        self.bits = self.setBits()
        self.binary = self.setBinary()
        self.decimal = self.decodeBinary()
       
    def setBits(self):
        return math.ceil(math.log2(20 * math.pow(10, self.precision)) + math.log2(1))
        
    def setBinary(self):
        return [random.randrange(0, 2) for row in range(self.bits)]
    
    def setDecimal(self):
        decimal = 0
        position = len(self.binary)
        for bit in self.binary:
            position -= 1
            decimal += bit * pow(2, position)
        return decimal
    
    def decodeBinary(self):
        return -10 + self.setDecimal() * 20 / (math.pow(2, self.bits) - 1)

## Implementacja osobnika o dwóch chromosomach

#### o __init__ - konstruktor
- `chromoPrec` - dokładność reprezentacji chromosomu, liczba całkowita
- `X` - chromosom X osobnika, obiekt typu Chromosome
- `Y` - chromosom Y osobnika, obiekt typu Chromosome
- `boothFuncValue` - wartość funkcji przystosowania osobnika, liczba zmiennoprzecinkowa

#### o boothFunc - implementacja wartości funkcji Booth osobnika
#### o reciprocalBoothFunc - implementacja odwotności wartości funkcji Booth osobnika

In [ ]:
class Individual:
    def __init__(self, chromoPrec):
        self.chromoPrec = chromoPrec
        self.X = self.setChromosome()
        self.Y = self.setChromosome()
        self.boothFuncValue = None
        
    def setChromosome(self):
        return Chromosome(self.chromoPrec)
    
    def setX(self, binary):
        self.X.binary = binary
        self.X.decimal = self.X.decodeBinary()
        
    def setY(self, binary):
        self.Y.binary = binary
        self.Y.decimal = self.Y.decodeBinary()
        
    def boothFunc(self):
        return pow((self.X.decimal + 2 * self.Y.decimal - 7), 2) + \
                pow((2 * self.X.decimal + self.Y.decimal - 5), 2)
        
    def setBoothFunc(self):
        self.boothFuncValue = self.boothFunc()
    
    def reciprocalBoothFunc(self):
        return 1 / self.boothFuncValue

## Implementacja populacji

#### o __init__ - konstruktor
- `chromoPrec` - dokładność reprezentacji chromosomów osobników populacji, liczba całkowita
- `size` - liczba osobnikow w populacji, liczba całkowita

####  o setPopulation - implementacja utworzenia populacji o rozmiarze `size`

#### o addIndividual - implementacja dodania osobnika do populacji
- `ind` - osobnik, obiekty klasy Individual

#### o getBest - odnalezienie osobnika najlepiej przystosowanego oraz jego wartości funkcji przystosowania  
#### o setPopulationBoothFunc - aktualizacja funkcji przystosowania wszystkich osobników populacji 
#### o getFitnessPopulation - pobranie listy wartości funkcji przystosowania osobników populacji 
#### o getMeanFitnessPopulation - pobranie średniej wartości funkcji przystosowania osobników populacji
#### o getStdFitnessPopulation - pobranie odchylenia standardowe wartości funkcji przystosowania osobników populacji
#### o calculateDistribution - obliczenie dystrybuanty osobników populacji

##### o elitarism - implementacja strategii elitarnej zgodnie z strategią z częściową reprodukcją poprzez elityzm
- `best` - osobniki o najlepszym przystosowaniu, obiekt klasy Population
- `percent` - procent populacji, liczba całkowita

In [ ]:
class Population:
    def __init__(self, size, chromoPrec):
        self.size = size
        self.chromoPrec = chromoPrec
        if size > 0:
            self.individuals = self.setPopulation()
        else :
            self.individuals = []
        
    def setPopulation(self):    
        return [Individual(self.chromoPrec) for row in range(self.size)]
    
    def addIndividual(self, ind):    
        self.individuals.append(ind)
        self.size += 1
        
    def getBest(self):
        bestVal = self.individuals[0].boothFuncValue
        bestInd = self.individuals[0]
        for ind in self.individuals:
            if ind.boothFuncValue < bestVal:
                bestVal = ind.boothFuncValue
                bestInd = ind
        return bestVal, bestInd
    
    def setPopulationBoothFunc(self):
        for ind in self.individuals:
            ind.setBoothFunc()
            
    def getFitnessPopulation(self):
        fitnessPop = []
        for ind in self.individuals:
            fitnessPop.append(ind.boothFuncValue)
        return fitnessPop
    
    def getMeanFitnessPopulation(self):
        mean = 0
        for ind in self.individuals:
            mean += ind.boothFuncValue
        return mean / self.size
    
    def getStdFitnessPopulation(self):
        mean = self.getMeanFitnessPopulation()
        variance = 0
        for ind in self.individuals:
            variance += ind.boothFuncValue - mean ** 2
        return (variance/self.size) ** 0.5
    
    def calculateDistribution(self):
        temp = [] 
        for ind in self.individuals:
            temp.append(ind.reciprocalBoothFunc())
        adaptSum = sum(temp)
        temp = [adaptation/adaptSum for adaptation in temp] 
        for i in range(1, len(temp)):
            temp[i] += temp[i-1]
        return temp
    
    def elitarism(self, best, percent):
        N = int(self.size * percent / 100) 
        for i in range(0, N):
            maxValue = self.individuals[0].boothFuncValue
            maxIndex = 0
            for j in range(1, self.size):
                if self.individuals[j].boothFuncValue > maxValue:
                    maxValue = self.individuals[j].boothFuncValue
                    maxIndex = j
            self.individuals[maxIndex].setX(best.individuals[i].X.binary)
            self.individuals[maxIndex].setY(best.individuals[i].Y.binary)
            self.setPopulationBoothFunc()

# Operatory genetyczne

### Selekcja

#### o bestSelection - implementacja selekcji najlepszych
- `pop` - populacja
- `percent` - procent najlepszych osobników, liczba całkowita
- `newPopulation` - najlepiej przystostowane osobniki z populacji, nowa populacja osobników

##### o calculateDistribution - implementacja wyznaczenia dystrybucji osobników dla danej populacji
- `pop` - populacja
- `temp` - dystrybucje osobników z populacji, lista
   
##### o rouletteWheelSelection - implementacja selekcji kołem ruletki
- `pop` - populacja
- `populationDist` - dystrybucja osobników populacji, lista
- `self.individuals[bestIndex]` - osobnik wybrany z populacji selekcją kołem ruletki, obiekt klasy Individual

In [ ]:
class Selection:
    def __init__(self, name, param = None):
        self.name = name
        if param is not None:
            self.param = param
            
    def bestSelection(self, pop):
        popAdaptation = []
        bestSelected = []
        N = int(pop.size * self.param / 100)

        for ind in pop.individuals:
            popAdaptation.append(ind.boothFuncValue)  

        for i in range(0, N):  
            best = min(popAdaptation)
            bestSelected.append(best)
            popAdaptation.remove(best) 

        newPopulation = Population(0, 8)
        for best in bestSelected:
            for ind in pop.individuals:
                if ind.boothFuncValue == best:
                    newPopulation.addIndividual(ind)
        return newPopulation
    
    def rouletteWheelSelection(self, pop):
        populationDist = pop.calculateDistribution()
        rand = random.random()
        previous = 0
        bestIndex = -1
        for index, value in enumerate(populationDist):
            if(previous <= rand <= value):
                bestIndex = index
            previous = value
        return pop.individuals[bestIndex]
            
    def select(self, pop):
        selected = Population(0, pop.chromoPrec)
        if self.name == 'najlepszych':  
            selected = self.bestSelection(pop) 
        if self.name == 'kołem ruletki':
            for i in range(pop.size):
                selected.addIndividual(self.rouletteWheelSelection(pop))   
        if self.name == 'turniejowa':
            for i in range(pop.size):
                selected.addIndividual(tournamentSelection(pop, self.param))
        return selected

#### o tournamentSelection - implementacja selekcji turniejowej
- `population` - populacja, obiekt klasy Population
- `k` -  wielkość turnieju, liczba całkowita
- `pop.individuals[0]` - osobnik wybrany z populacji selekcją turniejową, obiekt klasy Individual

In [ ]:
def tournamentSelection(population, k):
    pop = copy.deepcopy(population)
    popSize = len(pop.individuals)

    if popSize == 1:
        return pop.individuals[0]
    else:
        bits = pop.individuals[0].X.bits
        best = Population(0, bits)
        while(popSize > 0):
            popSize = len(pop.individuals)
            tournament = Population(0, bits)
            if popSize >= k:
                for i in range(k):
                    randInd = random.choice(pop.individuals)
                    tournament.addIndividual(randInd)
                    pop.individuals.remove(randInd)
            elif popSize > 1:
                tournament = copy.deepcopy(pop)    
                pop.individuals.clear() 
            else:
                break
            best.addIndividual(random.choice(tournament.individuals)) 
        return tournamentSelection(best, k)

### Krzyżowanie

#### Implementacja krzyżowania jedno, dwu i trzy punktowego oraz krzyżowania jednorodnego
- `name` - typ krzyżowania, string
- `prob` - prawdopodobieństwo krzyżowania, liczba z zakresu 0,1

##### o singlePoint, twoPoints, threePoints, probUniform - metody krzyżowania dwóch chromosomów
- `x1` - chromosom pierwszy, lista
- `x2` - chromosom drugi, lista

##### o singlePoint, twoPoints, threePoints, probUniform - metody krzyżowania dwóch osobników
- `ind1` - pierwszy osobnik, obiekt typu Individual
- `ind2` - pierwszy osobnik, obiekt typu Individual

##### crossPop - metoda krzyżowania populacji wybraną metodą
- `population` - populacja, obiekt typu Population

In [ ]:
class Crossover:
    def __init__(self, name, prob):
        self.name = name
        self.prob = prob
        
    def singlePoint(self, cr1, cr2):
        x1 = copy.deepcopy(cr1)
        x2 = copy.deepcopy(cr2)
        prob = random.random()
        if prob < self.prob:
            locus = random.randrange(1, len(x1))
            temp = x1[:locus]
            x1[:locus] = x2[:locus]
            x2[:locus] = temp
        return x1, x2
    
    def twoPoints(self, cr1, cr2):
        x1 = copy.deepcopy(cr1)
        x2 = copy.deepcopy(cr2)
        prob = random.random()
        if prob < self.prob:
            locuses = random.sample(range(1, len(x1)), 2)
            locus1 = min(locuses)
            locus2 = max(locuses)
            temp = x1[locus1:locus2]
            x1[locus1:locus2] = x2[locus1:locus2]
            x2[locus1:locus2] = temp
        return x1, x2
    
    def threePoints(self, cr1, cr2):
        x1 = copy.deepcopy(cr1)
        x2 = copy.deepcopy(cr2)
        prob = random.random()
        if prob < self.prob:
            locuses = random.sample(range(1, len(x1)), 3)
            locus1 = min(locuses)
            locus2 = max(locuses)
            locus3 = [i for i in locuses if i != locus1 and i != locus2][0]
            temp = x1[locus1:locus2]
            x1[locus1:locus2] = x2[locus1:locus2]
            x2[locus1:locus2] = temp
            temp = x1[locus3:]
            x1[locus3:] = x2[locus3:]
            x2[locus3:] = temp
        return x1, x2
    
    def probUniform(self, cr1, cr2):
        x1 = copy.deepcopy(cr1)
        x2 = copy.deepcopy(cr2)
        for index in range(len(x1)):
            prob = random.random()
            if prob < self.prob:
                temp = x1[index]
                x1[index] = x2[index]
                x2[index] = temp
        return x1, x2
    
    def getNewInd(self, chromoPrec, x1, x2, y1, y2):
        new1 = Individual(chromoPrec)
        new2 = Individual(chromoPrec)
        new1.setX(x1)
        new2.setX(x2)
        new1.setY(y1)
        new2.setX(y2)
        return new1, new2
    
    def singlePointInd(self, ind1, ind2):
        x1, x2 = self.singlePoint(ind1.X.binary, ind2.X.binary)
        y1, y2 = self.singlePoint(ind1.Y.binary, ind2.Y.binary)
        return self.getNewInd(ind1.chromoPrec, x1, x2, y1, y2)
        
    def twoPointsInd(self, ind1, ind2):
        x1, x2 = self.twoPoints(ind1.X.binary, ind2.X.binary)
        y1, y2 = self.twoPoints(ind1.Y.binary, ind2.Y.binary)
        return self.getNewInd(ind1.chromoPrec, x1, x2, y1, y2)
        
    def threePointsInd(self, ind1, ind2):
        x1, x2 = self.threePoints(ind1.X.binary, ind2.X.binary)
        y1, y2 = self.threePoints(ind1.Y.binary, ind2.Y.binary)
        return self.getNewInd(ind1.chromoPrec, x1, x2, y1, y2)
    
    def uniformInd(self, ind1, ind2):
        x1, x2 = self.probUniform(ind1.X.binary, ind2.X.binary)
        y1, y2 = self.probUniform(ind1.Y.binary, ind2.Y.binary)
        return self.getNewInd(ind1.chromoPrec, x1, x2, y1, y2)
        
    def crossPop(self, population):
        crossed = Population(0, population.chromoPrec)
        new1 = -1
        new2 = -1
        flag = True
        while flag:
            if len(population.individuals) == 0:
                flag = False
            elif len(population.individuals) == 1:
                new1 = population.individuals.pop(0)
                crossed.addIndividual(new1)
            else:
                choices = random.sample(range(len(population.individuals)), 2)
                choice1 = population.individuals.pop(max(choices))
                choice2 = population.individuals.pop(min(choices))    
                if self.name == 'jednopunktowe':  
                    new1, new2 = self.singlePointInd(choice1, choice2)
                elif self.name == 'dwupunktowe':
                    new1, new2 = self.twoPointsInd(choice1, choice2)
                elif self.name == 'trzypunktowe':
                    new1, new2 = self.threePointsInd(choice1, choice2)
                elif self.name == 'jednorodne':
                    new1, new2 = self.uniformInd(choice1, choice2)
                crossed.addIndividual(new1)
                crossed.addIndividual(new2)
        return copy.deepcopy(crossed)

### Mutacja 

- `name` - typ mutacji, string
- `prob` - prawdopodobieństwo mutacji, liczba z zakresu 0,1

#### o onePointMutation, twoPointsMutation, edgeMutation -  implementacja mutacji jedno i dwupunktowej, brzegowej

#### o inversion - implementacja operatora inwersji

#### o mutate - mutacja chromosomu wybraną metodą
- `chromo` - chromosom, obiekt klasy Chromosome

#### o mutatePop - mutacja populacji wybraną metodą
- `pop` - populacji, obiekt klasy Population

In [ ]:
class Mutation:
    def __init__(self, name, prob):
        self.name = name
        self.prob = prob
            
    def onePointMutation(self, chromo, mutationProb):
        prob = random.random()
        if prob < mutationProb:
            point = random.randrange(0, chromo.bits)
            chromo.binary[point] = 1 if chromo.binary[point] == 0 else 0
        chromo.decimal = chromo.decodeBinary()
        
    def twoPointsMutation(self, chromo, mutationProb):
        points = random.sample(range(0, chromo.bits), 2)
        for point in points:
            prob = random.random()
            if prob < mutationProb:
                chromo.binary[point] = 1 if chromo.binary[point] == 0 else 0
        chromo.decimal = chromo.decodeBinary()
        
    def edgeMutation(self, chromo, mutationProb):
        points = [0, chromo.bits - 1]
        for point in points:
            prob = random.random()
            if prob < mutationProb:
                chromo.binary[point] = 1 if chromo.binary[point] == 0 else 0
        chromo.decimal = chromo.decodeBinary()
        
    def inversion(self, chromo, inversionProb):
        prob = random.random()
        if prob < inversionProb:
            disables = True
            while(disables):
                points = random.sample(range(1, chromo.bits), 2)
                point1 = min(points)
                point2 = max(points)
                disables = False if (point2 >= point1 + 2) else True
            inversion = chromo.binary[point1:point2]
            inversion.reverse()
            chromo.binary = chromo.binary[:point1] + inversion + chromo.binary[point2:] 
        chromo.decimal = chromo.decodeBinary()
            
    def mutate(self, chromo):
        if self.name == 'jednopunktowa':  
            self.onePointMutation(chromo, self.prob)
        if self.name == 'dwupunktowa':
            self.twoPointsMutation(chromo, self.prob)
        if self.name == 'brzegowa':
            self.edgeMutation(chromo, self.prob)
        if self.name == 'inwersja':
            self.inversion(chromo, self.prob)
            
    def mutatePop(self, pop):
        for ind in pop.individuals:
            self.mutate(ind.X)
            self.mutate(ind.Y)

## Algorytm genetyczny celem minimalizacji funkcji

- `popSize` - liczba osobników populacji, liczba całkowita
- `chromoPrec` -  dokładność reprezentacji chromosomów, liczba całkowita
- `epochs` - liczba epok (generacji), liczba całkowita
- `sel` - typ selekcji, string
- `selParam` - parametr selekcji, liczba całkowita
- `cr` - typ krzyżowania, string
- `crP` - prawdopodobieństwo krzyżowania, liczba zmiennoprzecinkowa od 0 do 1
- `mut` - typ mutacji, string
- `mutP` - prawdopodobieństwo mutacji, liczba zmiennoprzecinkowa od 0 do 1
- `elitP` - procent osobników w strategii elitarnej, liczba całkowita

#### o findMinimum - implementacja algortymu genetycznego 
- `(round(bestSolution.X.decimal, 5), round(bestSolution.Y.decimal,5))` - punkt uzyskanego minimum, tupla
- `best[-1]` - uzyskana wartość minimalna, liczba zmiennoprzecinkowa
- `minEp` - wartość funkcji celu w kolejnych epokach, lista
- `meanEp` - średnia wartość funkcji celu w kolejnych epokach, lista
- `stdEp` - odchylenie standardowe w kolejnych epokach, lista

In [ ]:
class Optimizer:
    def __init__(self, popSize, epochs, chromoPrec, sel, selParam, cr, crP, mut, mutP, elitP): 
        self.popSize = popSize
        self.epochs = epochs
        self.chromoPrec = chromoPrec
        self.sel = sel
        self.selParam = selParam
        self.cr = cr
        self.crP = crP
        self.mut = mut
        self.mutP = mutP
        self.elitP = elitP
    
    def findMinimum(self):
        population = Population(self.popSize, self.chromoPrec)

        bestFitness = None
        bestSolution = None
        best = []
        bestGenerationNumber = -1 
        minEp = []  
        meanEp = []
        stdEp = []

        selection = Selection(self.sel, self.selParam)
        crossover = Crossover(self.cr, self.crP)
        mutation = Mutation(self.mut, self.mutP)

        for i in range(self.epochs):
            population.setPopulationBoothFunc()
            bestValue, bestInd = population.getBest()

            minEp.append(bestValue)
            meanEp.append(population.getMeanFitnessPopulation())
            stdEp.append(population.getStdFitnessPopulation())

            if(len(best) == 0 or best[-1] > bestValue):
                best.append(bestValue)
                bestSolution = bestInd
                bestGenerationNumber = i

            if(self.elitP > 0):
                bestSelection = Selection('najlepszych', self.elitP)
                elitaryPop = bestSelection.select(population)
           
            population = selection.select(population)
            population = crossover.crossPop(population)
            mutation.mutatePop(population)
            population.setPopulationBoothFunc()
            
            if(self.elitP > 0):
                population.elitarism(elitaryPop, self.elitP)
            
        return (round(bestSolution.X.decimal, 5), round(bestSolution.Y.decimal,5)), best[-1], minEp, meanEp, stdEp

### Interfejs graficzny - główna metoda to initApp

In [ ]:
class App():
    def __init():
        pass
    
    def initButton(self, desc, color, width, height):
        return ToggleButton(description = desc, disabled = False, button_style = color, layout = Layout(width = width, height = height))

    def initHbox(self, text, maximum, step, val):
        return HBox([Label(text, layout = Layout(width = '22%')), 
            IntSlider(min = 0, max = maximum, step = step, value = val, layout = Layout(width = '50%'))])

    def initFloatHbox(self, text, maximum, step, val):
        return HBox([Label(text, layout = Layout(width = '22%')), FloatSlider(min = 0, max = maximum, step = step, value = val, layout = Layout(width = '50%'))])
    
    def initConfPanel(self):
        global populationSl, bitsSl, epochSl
        confHeader = self.initButton('Konfiguracja', 'danger', '72%', '50px')
        populationSl = self.initHbox('Liczba osobników populacji', 200, 1, 50)
        bitsSl = self.initHbox('Dokłandność chromosomów', 20, 1, 6)
        epochSl = self.initHbox('Liczba epok', 500, 1, 250)
        confSliders = Box(children = [populationSl, bitsSl, epochSl], layout = Layout(display = 'flex', flex_flow = 'column', padding = "15px"))
        return AppLayout(header = confHeader, center = confSliders)
    
    def initGenPanel(self):
        global selectionBtn, crossoverBtn, mutationBtn, selectionSl, crossoverSl, mutationSl, elitarySl
        
        genHeader = self.initButton('Operatory genetyczne', 'danger', '72%', '50px')
        selectionBtn = ToggleButtons(options = ['turniejowa', 'kołem ruletki'], description = 'Selekcja: ', button_style = 'danger')
        crossoverBtn = ToggleButtons(options = ['jednopunktowe', 'dwupunktowe', 'trzypunktowe', 'jednorodne'], description = 'Krzyżowanie: ', button_style = 'warning')
        mutationBtn = ToggleButtons(options = ['jednopunktowa', 'dwupunktowa', 'brzegowa', 'inwersja'],description = 'Mutacja: ', button_style = 'success')

        selectionSl = self.initHbox('Wielkość turnieju', 100, 1, 3) 
        crossoverSl = self.initFloatHbox('Prawdopodobieństwo krzyżowania', 1, 0.01, 0.9)
        mutationSl = self.initFloatHbox('Prawdopodobieństwo mutacji', 1, 0.01, 0.2)
        label = Label('Określ procent osobników, które jako najlepiej przystosowane z populacji początkowej przejdą do populacji potomnej:')
        elitarySl = self.initHbox('Strategia elitarna (%)', 100, 1, 0) 

        buttons = Box(children = [selectionBtn, crossoverBtn, mutationBtn], layout = Layout(display = 'flex', flex_flow = 'column'))
        geneticSliders = Box(children = [selectionSl, crossoverSl, mutationSl, label, elitarySl], layout = Layout(display = 'flex', flex_flow = 'column', padding = "15px"))
        selectionBtn.observe(self.selectionBtnClicked, 'value')
        box = Box(children = [buttons, geneticSliders], layout = Layout(display = 'flex', flex_flow = 'column'))

        return AppLayout(header = genHeader, center = box)

    def selectionBtnClicked(self, change):
        if change['new'] == 'kołem ruletki':
            selectionSl.layout.display = 'none'
        if change['new'] == 'turniejowa':
            selectionSl.layout.display = 'block'
            selectionSl.layout.display = 'flex'
            selectionSl.layout.flex_flow = 'row'
            
    def getResults(self):
        popSize = populationSl.children[1].value
        chromoPrec = bitsSl.children[1].value
        epoch = epochSl.children[1].value
        sel = selectionBtn.value
        cr = crossoverBtn.value
        mut = mutationBtn.value
        elitary = elitarySl.children[1].value
        if sel != 'kołem ruletki':
            selParam = selectionSl.children[1].value
        else: 
            selParam = 0
        crProb = crossoverSl.children[1].value
        mutProb = mutationSl.children[1].value  

        self.updateConfInfo(popSize, chromoPrec, epoch, sel, selParam, cr, crProb, mut, mutProb, elitary)
        opt = Optimizer(popSize, epoch, chromoPrec, sel, selParam, cr, crProb, mut, mutProb, elitary)
        
        start = time.time()
        xy, minimum, minEp, meanEp, stdEp = opt.findMinimum()
        end = time.time()

        self.updateResults(round(end - start, 5), 'sekund', round(minimum, 5), xy)
        self.setDataPlot1(minEp)
        self.setDataPlot2(meanEp)
        self.setDataPlot3(stdEp)
        
        app.children[1].layout.display = 'none'
        app.children[2].layout = Layout(display = 'flex', flex_flow = 'row')

    def startApp(self, change):
        app.children[0].layout.display = 'none'
        app.children[1].layout.display = 'block'
        time.sleep(0.1)
        self.getResults()
        
    def initFirstPanel(self):
        global startBtn
        configLayout = self.initConfPanel()
        geneticLayout = self.initGenPanel()
        startBtn = self.initButton('Rozpocznij', 'success', '72%', '50px')
        return Box(children = [configLayout, geneticLayout, startBtn], layout = Layout(display = 'flex', flex_flow = 'column'))  
        
    def showPlot(self, data, ylabel):
        plt.figure(figsize = (20, 30))
        plt.plot(data, color = 'red')
        plt.xlabel('Epoka')
        plt.ylabel(ylabel)
        plt.show()

    def showFirstPlot(self, value):
        if value['new'] == True:
            self.showPlot(self.getDataPlot1(), 'Wykres zależności wartości funkcji celu od kolejnej iteracji')
        boothFuncBtn.value = False

    def showSecondPlot(self, value):
        if value['new'] == True:
            self.showPlot(self.getDataPlot2(), 'Wykres zależności średniej wartości funkcji celu od kolejnej iteracji')
        meanBoothFuncBtn.value = False
    
    def showThreePlot(self, value):
        if value['new'] == True:
            self.showPlot(self.getDataPlot3(), 'Wykres zależności odchylenia standardowego od kolejnej iteracji')
        stdBtn.value = False

    global dataPlot1
    def setDataPlot1(self, newData):
        global dataPlot1
        dataPlot1 = newData

    def getDataPlot1(self):
        global dataPlot1
        return dataPlot1 if 'dataPlot1' in globals() else []

    global dataPlot2
    def setDataPlot2(self, newData):
        global dataPlot2
        dataPlot2 = newData

    def getDataPlot2(self):
        global dataPlot2
        return dataPlot2 if 'dataPlot2' in globals() else []

    global dataPlot3
    def setDataPlot3(self, newData):
        global dataPlot3
        dataPlot3 = newData

    def getDataPlot3(self):
        global dataPlot3
        return dataPlot3 if 'dataPlot3' in globals() else []
    
    def restartApp(self, change):
        app.children[2].layout.display = 'none'
        app.children[0].layout =  Layout(display = 'flex', flex_flow = 'column')
    
    def saveFile(self, value):
        if value['new'] == True:
            results = { 
                'wartości funkcji celu' : self.getDataPlot1(),
                'średnie wartości funkcji celu' : self.getDataPlot2(),
                'odchylenie standardowe' : self.getDataPlot3(),
            } 
            path = fileInput.children[1].value + '.csv'
            df = pd.DataFrame(results)
            try:
                df.to_csv(path)
            except Exception as f:
                f.write('an error message')
        fileButton.value = False
    
    def initResultPanel(self):
        global infoDesc, resultsArea, boothFuncBtn, meanBoothFuncBtn, stdBtn, fileInput, fileButton, returnBtn

        params = self.initButton('Wybrane parametry', '', '100%', '50px')
        infoDesc = Textarea(value = '', disabled = True, layout = Layout(width = "400px", height = "290px"))

        resultsHeader = self.initButton('Uzyskane rezultaty', '', '100%', '50px')
        resultsArea = Textarea(value = '', disabled = True, layout = Layout(width = "480px", height = "90px"))

        boothFuncBtn = self.initButton('Wykres zależności wartości funkcji celu od kolejnej iteracji', 'danger', '100%', '') 
        meanBoothFuncBtn = self.initButton('Wykres zależności średniej wartości funkcji celu od kolejnej iteracji', 'danger', '100%', '')
        stdBtn = self.initButton('Wykres zależności odchylenia standardowego od kolejnej iteracji', 'danger', '100%', '') 
        plotsBox = Box(children = [boothFuncBtn, meanBoothFuncBtn, stdBtn], layout = Layout(display = 'flex', flex_flow = 'column', margin = "10px"))

        fileInput = HBox([Label('Zapis do pliku:', layout = Layout(width = '180px')),  Text(value = '', placeholder = 'lokalizacja pliku', disabled = False, layout = Layout(width = '80%cd'))])
        fileButton = self.initButton('Zapisz', 'warning', '20%', '')
        fileBox = Box(children = [fileInput, fileButton], layout = Layout(display = 'flex', flex_flow = 'row', margin = "10px"))
        
        returnBtn = self.initButton('Skonfiguruj ustawienia algorytmu ponownie', 'success', '100%', '')

        left = VBox(children = [params, infoDesc])
        right = VBox(children = [resultsHeader, resultsArea, plotsBox, fileBox, returnBtn])

        boothFuncBtn.observe(self.showFirstPlot, 'value')
        meanBoothFuncBtn.observe(self.showSecondPlot, 'value')
        stdBtn.observe(self.showThreePlot, 'value')
        returnBtn.observe(self.restartApp, 'value')
        fileButton.observe(self.saveFile, 'value')

        return Box([left, right], layout = Layout(display = 'none'))
    
    def updateConfInfo(self, popSize, prec, epoch, selection, selParam, crossover, crProb, mutation, mutProb, elitary):
        out = 'Liczba osobników populacji: ' + str(popSize) + '\nDokładność reprezentacji chromosomów: ' + str(prec) + \
            '\nLiczba epok: ' + str(epoch) + '\n\nSelekcja: ' + selection
        if selection == 'najlepszych' or selection == 'strategia elitarna':
            out += '\nProcent osobników: ' + str(selParam)
        if selection == 'turniejowa':
            out += '\nWielkość turnieju: ' + str(selParam)    
        out += '\n\nKrzyżowanie: ' + crossover + '\nPrawdopodobieństwo krzyżowania: ' + str(crProb) + \
            '\n\nMutacja: ' + mutation + '\nPrawdopodobieństwo mutacji: ' + str(mutProb) + \
            '\n\nProcent w strategii elitarnej: ' + str(elitary) 
        
        infoDesc.value = out

    def updateResults(self, time, timeUnit, minimum, xy):
        resultsArea.value = 'Czas wykonania: ' + str(time) + ' ' + timeUnit + '\n\nWynik minimalizacji funkcji: ' + str(minimum) + '\nW punkcie: ' + str(xy)
    
    def initApp(self): 
        global app
        app = Box(children = [self.initFirstPanel(), Label('Calculating...'), self.initResultPanel()], layout = Layout(display = 'flex', flex_flow = 'column'))
        startBtn.observe(self.startApp, 'value')
        app.children[1].layout.display = 'none'
        return app

In [ ]:
%matplotlib qt

In [ ]:
App().initApp()